# 4 Intro to PyLabRobot: Writing protocols that interface with Peripherals

### Learning Objectives:
**4.0.1** Instantiating a Dummy Peripheral

**4.0.2** Moving Plates onto Peripherals

**4.0.3** Generating Dummy Data

**4.0.4** Data Wrangling

**4.0.5** Using data as control for a liquid handling protocol

### Imports

In [53]:
# Importing necessary modules from the PyLabRobot package to handle liquids and␣visualize processes.
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends import ChatterBoxBackend
from pylabrobot.visualizer.visualizer import Visualizer

# This import provides access to OTDeck, which represents the deck of an␣Opentrons robot.
from pylabrobot.resources.opentrons import OTDeck
# Loading and Plate Management

# Imports functions and classes to load resources and manage plate types␣specific to Opentrons robots.
from pylabrobot.resources.opentrons.load import *
from pylabrobot.resources.opentrons.plates import *

# Tracking and Contamination Prevention
# Enables tracking of tip usage, liquid volume, and helps prevent␣cross-contamination in experiments.
from pylabrobot.resources import set_tip_tracking, set_volume_tracking, set_cross_contamination_tracking
from pylabrobot.resources.plate import *

# Optional, use when interested in tracking the state of tips and volumes, generally keep this on
set_tip_tracking(True), set_volume_tracking(True)

# Optional, use when interested in protecting against accidental cross contamination
set_cross_contamination_tracking(True)

# External Libraries
# Importing standard libraries for additional functionality.
import opentrons # Provides access to Opentrons API for robot control.
import time # Allows for adding delays in the robot's operation for timing␣experiments.
import pandas as pd
import numpy as np

#### Labware

In [49]:
from pylabrobot.resources import (
    opentrons_24_tuberack_eppendorf_2ml_safelock_snapcap,
    opentrons_96_tiprack_300ul,
    corning_96_wellplate_360ul_flat,
    corning_384_wellplate_112ul_flat
)

### 4.0.1 Instantiating a Dummy Peripheral 
#### If you ever get stuck, click the circley arrow at the top of the notebook to restart your progress!

`Peripherals` are machines that are wired into a liquid handling set-up. They can really be anything, but for the sake of this tutorial, we will be using a plate reader. To make the `Peripheral` accessible by our code, we will make a dummy class. Use the following code as inspiration should you wish to make a `Peripheral` for a different machine. The general idea is that the `Peripheral` is an object that produces **formatted data**. Most times, you will have to then perform some data processing to format the data correctly, and then apply a line of best fit to get the quantity of interest, as most relevant quantities aren't directly measurable.

In [69]:
class PlateReader:
    def __init__(self):
        pass

    def generate_data(self, plate, filename="data.csv", mean_absorbance=0.5, std_dev=0.1):
        # Get rows and columns from the plate
        rows = plate.num_items_y
        cols = plate.num_items_x

        # Sample a normal distribution to generate data
        data = []
        for row in range(rows):
            row_data = []
            for col in range(cols):
                absorbance_value = np.random.normal(loc=mean_absorbance, scale=std_dev)
                row_data.append(absorbance_value)
            data.append(row_data)

        # Write all generated data to a csv file
        df = pd.DataFrame(data, index=[chr(65 + i) for i in range(rows)], columns=[str(i+1) for i in range(cols)])
        df.to_csv(filename, index=True, header=True)
        
        print(f"Wrote data to {filename}!")

# Declare a PlateReader()
reader = PlateReader()

In [83]:
# We'll demonstrate functionality with both a 96 and 384 well plate
plate = corning_96_wellplate_360ul_flat("plate")
plate_384 = corning_384_wellplate_112ul_flat("384_plate")

### 4.0.2-4 Interfacing with the Peripheral and Acquiring Data
Since the peripherals aren't implemented in the visualizer yet, we won't be able to see the movement of the plates to the PlateReader. That's okay, though. We can use our imaginations and pass in the plate to our reader. 

One of the key ideas of using the `PlateReader` is that it will only give us a raw signal. We need to use a `line of best fit` to relate the raw absorbance read to the quantity we care about, Concentration.

In [70]:
# Let's pass our 96-well plate into the reader and acquire some data!
reader.generate_data(plate=plate, filename="data_96.csv")

Wrote data to data_96.csv!


In [71]:
# It also works for 384-well plates!
reader.generate_data(plate=plate_384, filename="data_384.csv")

Wrote data to data_384.csv!


In [79]:
# This command will read the .csv file so we can visualize our data,
# Recall, these numbers are ABSORBANCE, NOT CONCENTRATION
print("Raw Absorbance")
pd.read_csv("data_96.csv", index_col=0)

Raw Absorbance


,1,2,3,4,5,6,7,8,9,10,11,12
A,0.467541,0.413883,0.540936,0.461422,0.477619,0.474872,0.610570,0.352844,0.451773,0.442475,0.336408,0.529250
B,0.708349,0.512465,0.398305,0.506533,0.652430,0.553133,0.603775,0.659649,0.416051,0.467699,0.490868,0.505993
C,0.517879,0.335291,0.546290,0.274819,0.635270,0.532243,0.399472,0.704949,0.443658,0.497967,0.446935,0.664695
D,0.476399,0.466914,0.361236,0.578846,0.721405,0.598214,0.462446,0.490099,0.637163,0.326344,0.668525,0.485664
E,0.564242,0.595694,0.355615,0.557662,0.620570,0.509837,0.681745,0.531404,0.496665,0.649355,0.408988,0.359425
F,0.369354,0.519782,0.461623,0.441542,0.537902,0.434699,0.561449,0.310026,0.408100,0.456825,0.572971,0.635587
G,0.600390,0.520363,0.543447,0.397026,0.526905,0.369639,0.353402,0.534155,0.410244,0.281041,0.414980,0.699159
H,0.342316,0.658661,0.625080,0.471457,0.467598,0.410125,0.425904,0.334016,0.445521,0.403707,0.424437,0.427232


In [ ]:
pd.read_csv("data_384.csv", index_col=0)

We need to use a `line of best fit` to relate absorbance to concentration. This is an experimentally determined equation, but for the tutorial we will use a dummy line.

In [74]:
# Linear equation parameters (example values)
slope = 100  # Example slope
intercept = 0  # Example intercept

def read_and_calculate_concentrations(filename, slope, intercept):
    df = pd.read_csv(filename, index_col=0)
    for row in df.index:
        for col in df.columns:
            absorbance = df.at[row, col]
            concentration = absorbance * slope + intercept
            df.at[row, col] = concentration
    return df

# Process the data
conc_df = read_and_calculate_concentrations("data_96.csv", slope, intercept)

In [81]:
# Now we have a dataframe of the concentration of sample in each well. Recall we want to dilute all wells to 20 ng/uL
print("Concentration of Sample (ng/uL)")
conc_df

Concentration of Sample (ng/uL)


,1,2,3,4,5,6,7,8,9,10,11,12
A,46.754129,41.388260,54.093624,46.142192,47.761938,47.487226,61.057005,35.284392,45.177269,44.247517,33.640843,52.924961
B,70.834920,51.246547,39.830476,50.653283,65.243025,55.313285,60.377539,65.964879,41.605056,46.769940,49.086768,50.599345
C,51.787887,33.529130,54.629031,27.481947,63.527035,53.224329,39.947226,70.494856,44.365788,49.796693,44.693541,66.469503
D,47.639914,46.691392,36.123574,57.884645,72.140457,59.821395,46.244577,49.009935,63.716314,32.634437,66.852491,48.566366
E,56.424202,59.569446,35.561489,55.766163,62.056964,50.983665,68.174527,53.140372,49.666479,64.935455,40.898801,35.942541
F,36.935397,51.978206,46.162284,44.154166,53.790202,43.469874,56.144934,31.002603,40.809997,45.682459,57.297072,63.558725
G,60.038991,52.036275,54.344718,39.702622,52.690485,36.963898,35.340176,53.415476,41.024426,28.104051,41.498040,69.915928
H,34.231635,65.866149,62.507984,47.145735,46.759757,41.012525,42.590411,33.401647,44.552109,40.370653,42.443706,42.723190


### 4.0.5 Using data from the Peripheral as control for an experiment
We now have a dataframe of the concentrations of our sample in each well. It is a common operation to take a plate of randomly distributed samples and do a dilution to make everything a uniform concentration.

Recall the dilution equation $C_{1}V_{1} = C_{2}V_{2}$

In [82]:
def calculate_water_volume(target_conc, initial_volume, initial_conc):
    final_volume = (initial_conc * initial_volume) / target_conc
    volume_of_water = final_volume - initial_volume
    return volume_of_water

def apply_dilution(df, target_conc, initial_volume):
    dilution_df = df.copy()
    for row in df.index:
        for col in df.columns:
            initial_conc = df.at[row, col]
            volume_of_water = calculate_water_volume(target_conc, initial_volume, initial_conc)
            dilution_df.at[row, col] = volume_of_water
    return dilution_df

# Target final concentration and initial volume
target_conc = 20  # ng/uL
initial_volume = 50  # uL

water_to_add = apply_dilution(conc_df, target_conc, initial_volume)
print("Volume of water to add (uL)")
water_to_add

Volume of water to add (uL)


,1,2,3,4,5,6,7,8,9,10,11,12
A,66.885322,53.470649,85.234060,65.355479,69.404844,68.718064,102.642514,38.210980,62.943173,60.618793,34.102106,82.312402
B,127.087300,78.116369,49.576189,76.633206,113.107562,88.283214,100.943847,114.912197,54.012639,66.924851,72.716921,76.498363
C,79.469719,33.822826,86.572577,18.704868,108.817587,83.060823,49.868064,126.237141,60.914469,74.491731,61.733852,116.173758
D,69.099785,66.728479,40.308934,94.711612,130.351143,99.553488,65.611442,72.524838,109.290785,31.586093,117.131227,71.415914
E,91.060505,98.923615,38.903723,89.415407,105.142410,77.459162,120.436319,82.850930,74.166199,112.338637,52.247002,39.856354
F,42.338493,79.945516,65.405711,60.385414,84.475505,58.674684,90.362335,27.506508,52.024993,64.206147,93.242680,108.896813
G,100.097477,80.090687,85.861795,49.256554,81.726212,42.409744,38.350439,83.538689,52.561066,20.260129,53.745099,124.789821
H,35.579087,114.665373,106.269959,67.864336,66.899394,52.531314,56.476027,33.504117,61.380272,50.926633,56.109266,56.807976


### Exercises:

#### Exercise 1: Given that you have a dataframe of water volumes to add, write a liquid handling protocol that dilutes the 96-well plate to the target concentration of 20 ng/uL. Add labware to the deck as needed, including diluent and tip racks. The max volume of water to be added should not exceed the max capacity of the wells of the plate you chose.

#### Exercise 2: Perform the same work flow for the 384 plate. If you coded your solution to Exercise 1 in a Pythonic manner using functions, this should be trivial. Make the target concentration 40 ng/uL. Add labware to the deck as needed, including diluent and tip racks. The max volume of water to be added should not exceed the max capacity of the wells of the plate you chose.